In [2]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from joblib import dump, load

# source ./venv/Scripts/activate

file_name = 'prep_dataset.csv'
target_variable = "time_to_fail"

# Laden der präparierten Daten
df = pd.read_csv(file_name)

# Funktion zur Bewertung der Modelle
def evaluate_model(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true / 1000 / 3600 / 24, y_pred / 1000 / 3600 / 24)
    r2 = r2_score(y_true, y_pred)
    print(f"{model_name} - MSE: {mse:.2f}, R²: {r2:.2f}")

# Funktion zum Laden der Modelle
def load_model_if_exists(model_file):
    if os.path.exists(model_file):
        return load(model_file)
    else:
        return None


# Modelle laden
grid_search_file = "best_random_forest_model.joblib"
grid_search = load_model_if_exists(grid_search_file)
""""
Für die Vorhersage wird ein zwei-Phasen-Modell verwendet. Es beinhaltet zwei MachineLearning-Modelle.
Zunächst wird der Wert der Spalte 'Survival' mit einem Klassifikationsmodell von RandomForest vorhergesagt. Falls dieser Wert False ist, so geht das Modell davon aus,
dass die Komponente in der Zukunft einen Wert "VALID" in der Spalte 'status_75' haben, also ausfallen wird.
Wenn dies der Fall ist, dann werden die Daten in das zweite Modell, ein Regressionsmodell von RandomForest eingegeben.
Dies sagt den Wert der Spalte 'time_to_fail' vorher. Es wird also berechnet, wann(seit heute) die Komponente (vorraussichtlich) ausfallen wird.
Die Einheit der Spalte ist in 1/10- Sekunden.
Wenn in der ersten Phase ein Wert von True vorhergesagt wird, so ist es nicht sinnvoll, weiter vorherzusagen, wann die Komponente ausfallen wird,
wenn überhaupt nicht angenommen wird, dass diese ausfällt.
"""

'"\nFür die Vorhersage wird ein zwei-Phasen-Modell verwendet. Es beinhaltet zwei MachineLearning-Modelle.\nZunächst wird der Wert der Spalte \'Survival\' mit einem Klassifikationsmodell von RandomForest vorhergesagt. Falls dieser Wert False ist, so geht das Modell davon aus,\ndass die Komponente in der Zukunft einen Wert "VALID" in der Spalte \'status_75\' haben, also ausfallen wird.\nWenn dies der Fall ist, dann werden die Daten in das zweite Modell, ein Regressionsmodell von RandomForest eingegeben.\nDies sagt den Wert der Spalte \'time_to_fail\' vorher. Es wird also berechnet, wann(seit heute) die Komponente (vorraussichtlich) ausfallen wird.\nDie Einheit der Spalte ist in 1/10- Sekunden.\nWenn in der ersten Phase ein Wert von True vorhergesagt wird, so ist es nicht sinnvoll, weiter vorherzusagen, wann die Komponente ausfallen wird,\nwenn überhaupt nicht angenommen wird, dass diese ausfällt.\n'

In [3]:
# Load Regression Data
# Daten für Regression werden in Einflussvariable X1 und Zielvariable y aufgeteilt
# Es werden nur Datensätze betrachtet, die in Spalte 'Survival' den Wert "False" haben, da
# andernfalls der Wert von 'time_to_fail', also die Zeit bis zum Ausfall keinen Sinn ergibt, da
# es keinen Ausfall geben wird
y = df[target_variable]
X = df.drop(columns=[target_variable])
# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
# Regressionsmodell wird geladen, sofern vorhanden
rf_model_file = "random_forest_model.joblib"
rf_model = load_model_if_exists(rf_model_file)
print(df.columns.tolist())

['InvSampTime', 'RadV_50per', 'OfsDevTight', 'InvSampTurn', 'AmpDevStrict', 'RadV_25per', 'RawRadius', 'AmpSyncCheck', 'msg_timestamp', 'OfsCos200', 'OfsCos100', 'ClCos300', 'OfsSin200', 'ClSin300', 'time_to_fail', 'mileage', 'ClSin150', 'ClCos150', 'OfsSin100', 'EcuLifeT', 'sick_time']


In [11]:
# Random Forest Regressor
# Regressionsmodell wird berechnet und in Datei gespeichert, sofern nicht vorhanden
if rf_model is None:
    rf_model = RandomForestRegressor(n_estimators=50, random_state=42)
    rf_model.fit(X_train, y_train)
    dump(rf_model, rf_model_file)
# Modell wird mithilfe des MeanSquaredErrors und des R²- Wertes ausgewertet (möglichst geringer MSE und möglichst hoher R²- Wert ist optimal)
rf_predictions = rf_model.predict(X_test)
evaluate_model(
    y_test, rf_predictions, "Random Forest"
)  # MSE: 38.68, R²: 0.97 -> Durchschnittlich ein Fehler von +-6 Tagen

KeyboardInterrupt: 

In [ ]:
# Hyperparameter-Optimierung von RFR mit RandomSearchCV
# Für genaueres Modell:
# Größeren Trainingsdatensatz benutzen
# Hyperparameteroptimierung des RandomForest-Modells
if grid_search is None:
    param_grid = {
    "n_estimators": [50, 100, 200, 300],
    "max_depth": [None, 5, 10, 15, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4, 6],
    "max_features": ["auto", "sqrt", "log2"],
    }

    # GridSearchCV
    grid_search = GridSearchCV(
        estimator=rf_model,
        param_grid=param_grid,
        cv=5,  # Anzahl der Folds in der Kreuzvalidierung
        n_jobs=-2,  # Verwende alle verfügbaren Kerne
        verbose=2,
        scoring="neg_mean_squared_error",
    )

    # Fit des Modells
    grid_search.fit(X_train, y_train)

    # Beste Parameter und Modell
    best_rf_model = grid_search.best_estimator_
    dump(best_rf_model, grid_search_file)
best_rf_predictions = best_rf_model.predict(X_test)
evaluate_model(y_test, best_rf_predictions, "Best Random Forest")